### Inicializando o Navegador

In [189]:
#criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)


#importar/ visualizar a base de dados
tabela = pd.read_excel(r'buscas.xlsx')
display(tabela)




,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,2000,2700
1,rtx 3060,zota galax,1500,2000


### Definição das funções de busca no google shopping e no buscapé

In [190]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    #analisar se ele não tem termos banidos
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    #analisar se ele tem todos os termos do nome do produto
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False
    return tem_todos_termos_produtos



def busca_google_shopping(navegador, produto, termos_banidos, preco_maximo, preco_minimo): #toda variavel dentro da função tem que ser criada dentro da função ou passada para função como parametro
    
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_nome_produto = produto.split(' ')
    l_resultados =[]
    #entrar no google
    navegador.get('https://www.google.com.br/?hl=pt-BR')
    
    navegador.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)
    
    
    #entrar na aba shopping
    elementos = WebDriverWait(navegador, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[@role="listitem"]')))
    
    for elemento in elementos:
        if 'Shopping' in elemento.text:
            elemento.click()
            break
    #pegar as informações do produto
    lista_resultados = WebDriverWait(navegador, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'i0X6df')))
    
    for resultado in lista_resultados:
        
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        nome = nome.lower()
        
        #analisar se ele não tem termos banidos
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
        
        
        #analisar se ele tem todos os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)
        
            #selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produtos = True
        if not tem_termos_banidos  and tem_todos_termos_produtos:
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
                preco = float(preco)
            
                preco_minimo = float(preco_minimo)
                preco_maximo = float(preco_maximo)
            except:
                continue
            
            #verificar se o preco está entre o preço min e o preço max
            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = resultado.find_element(By.CLASS_NAME, 'bONr3b')
                elemento_pai = elemento_referencia.find_element(By.XPATH, '..')
                link = elemento_pai.get_attribute('href')
    
                l_resultados.append((nome, preco, link))
    return l_resultados


def busca_buscape(navegador, produto, termos_banidos, preco_maximo, preco_minimo):
    #tratar os valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_nome_produto = produto.split(' ')
    l_resultados =[]
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    
    #buscar no buscapé
    navegador.get('https://www.buscape.com.br/')
    navegador.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__WAC2Y').send_keys(produto, Keys.ENTER)

    #pegar os resultados
    lista_resultados = WebDriverWait(navegador, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ProductCard_ProductCard_Inner__gapsh')))

    for resultado in lista_resultados:
        preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__HEz7L').text
        nome = resultado.find_element(By.CLASS_NAME ,'ProductCard_ProductCard_Name__U_mUQ').text
        nome = nome.lower()
        link = resultado.get_attribute('href')
        #print(nome, preco, link)
    
        #analisar se ele não tem termos banidos
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
        
        
        #analisar se ele tem todos os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

        #analisar se o preco está na faixa de preço min e preço máx
        if not tem_termos_banidos  and tem_todos_termos_produtos:
            preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
            preco = float(preco)
            if preco_minimo <= preco <= preco_maximo:
                l_resultados.append((nome, preco, link))
        
        #retornar a lista de ofertad do buscapé
    return l_resultados

### Tabela de ofertas

In [191]:
tabela_ofertas = pd.DataFrame()

for linha in tabela.index:
    #pesquisar pelo produto
    produto = tabela.loc[linha, 'Nome']
    termos_banidos = tabela.loc[linha, 'Termos banidos']
    preco_minimo = tabela.loc[linha, 'Preço mínimo']
    preco_maximo = tabela.loc[linha, 'Preço máximo']
    
    lista_ofertas_google_shopping = busca_google_shopping(navegador, produto, termos_banidos, preco_maximo, preco_minimo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None
    lista_ofertas_buscape = busca_buscape(navegador, produto, termos_banidos, preco_maximo, preco_minimo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None
display(tabela_ofertas)

,produto,preco,link
0,seminovo iphone 12 64gb - preto - condição exc...,2599.00,https://www.google.com.br/url?url=https://www....
1,iphone 12 apple 64gb roxo - vitrine,2033.10,https://www.google.com.br/url?url=https://www....
2,seminovo iphone 12 64gb - branco - condição ex...,2239.00,https://www.google.com.br/url?url=https://www....
3,usado: iphone 12 64gb preto muito bom - trocaf...,2537.10,https://www.google.com.br/url?url=https://www....
4,"vitrine: iphone 12 apple 64gb branco tela 6,1 ...",2399.99,https://www.google.com.br/url?url=https://www....
5,iphone 12 64gb branco bom,2207.92,https://www.google.com.br/url?url=https://www....
6,iphone 12 64gb preto - semi novo,2024.10,https://www.google.com.br/url?url=https://mali...
7,usado: iphone 12 64gb preto muito bom - trocafone,2510.10,https://www.google.com.br/url?url=https://www....
8,usado: iphone 12 64gb preto bom - trocafone,2569.00,https://www.google.com.br/url?url=https://www....
9,vitrine iphone 12 azul 64gb,2159.99,https://www.google.com.br/url?url=https://www....


### Exportar para o excel

In [192]:
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

### Enviar por e-mail o resultado da tabela

In [193]:
import win32com.client as win32 #permite acessar programas da microsoft instalados no seu pc
outlook = win32.Dispatch('outlook.application') #abrir o outlook



In [194]:

if len(tabela_ofertas) > 0:
    mail = outlook.CreateItem(0) #cria o email
    mail.To = 'wollace2099@gmail.com' #remetente
    mail.Subject = 'Produtos encontrados na faixa de preço desejada' #Assunto
    mail.HTMLBody = f'''
    <p>Prezados,</p>
    <p>Encontramos alguns produtos na faixa de preço desejada</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Att., Wollace</p>''' #corpo do texto
    #ou mail.HTMLBody = '<p>Corpo do email em HTML <p>'
    
    mail.Send() #envia o email
    navegador.quit()